## Step 0: Imports

In [ ]:
from pinecone import Pinecone, ServerlessSpec
from sentence_transformers import SentenceTransformer
import os

In [ ]:
# Step 2: Import libraries
from pinecone import Pinecone, ServerlessSpec
from sentence_transformers import SentenceTransformer
import os

# Step 3: Initialize Pinecone
api_key = "your_pinecone_api"  # 🔐 Paste your Pinecone API key
pc = Pinecone(api_key=api_key)

# Step 4: Create index (only do this once, skip if already created)
index_name = "nlp-demo2"
dimension = 384  # because we're using all-MiniLM-L6-v2

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=dimension,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",        # or "gcp"
            region="us-east-1"  # match your Pinecone project region
        )
    )
model = SentenceTransformer("all-MiniLM-L6-v2")

## Step 1: Replace Sentences with FAQs

In [2]:
faqs = [
    {"title": "Reset Password", "text": "You can reset your password from the settings page.", "category": "account"},
    {"title": "Refund Policy", "text": "Refunds are processed within 5–7 business days.", "category": "billing"},
    {"title": "Upgrade Plan", "text": "You can upgrade your plan from the billing section.", "category": "billing"},
    {"title": "Contact Support", "text": "To contact support, go to the Help Center.", "category": "support"},
    {"title": "Free Trial", "text": "We offer a free trial for all new users.", "category": "pricing"},
]

## Step 2: Encode and Upsert with Metadata

In [3]:
# Extract only the text for embedding
texts = [item["text"] for item in faqs]
embeddings = model.encode(texts)

# Add metadata + upsert
to_upsert = [
    (
        f"id-{i}",
        embeddings[i].tolist(),
        {  # metadata dictionary
            "title": faqs[i]["title"],
            "category": faqs[i]["category"],
            "text": faqs[i]["text"]
        }
    )
    for i in range(len(faqs))
]
index = pc.Index(index_name)
index.upsert(vectors=to_upsert)

{'upserted_count': 5}

## Step 3: Search and Return Metadata

In [4]:
user_question = input("💬 Ask your question: ")
query_vec = model.encode([user_question]).tolist()[0]

result = index.query(vector=query_vec, top_k=2, include_metadata=True)

print(f"\n🔍 Question: {user_question}\n")
for match in result["matches"]:
    meta = match["metadata"]
    print(f"📌 Title: {meta['title']}")
    print(f"📂 Category: {meta['category']}")
    print(f"✅ Answer: {meta['text']}")
    print(f"🔢 Score: {match['score']:.4f}\n")

💬 Ask your question:  I need help contacting customer support.



🔍 Question: I need help contacting customer support.

📌 Title: Contact Support
📂 Category: support
✅ Answer: To contact support, go to the Help Center.
🔢 Score: 0.7232

📌 Title: Upgrade Plan
📂 Category: billing
✅ Answer: You can upgrade your plan from the billing section.
🔢 Score: 0.3551

